In [1]:
import requests
import json
import re
import urllib
import threading
from bs4 import BeautifulSoup

class Solution:
    
    def __init__(self):
        self.URL = URL_name
        self.headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"}
        self.df = dict()   

        
    def preprocessing(self):
        req = requests.get(url=self.URL, headers=self.headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        location = soup.find("script",attrs={"id":"initials","type":"application/json"})
        start_index=location.text.index("smartRecruiterResult")-1
        end_index=len(location.text)
        job_data="{" + location.text[start_index:end_index] #Slicing to extract only data related to job openings
        return job_data
        
    def url_extraction(self, job_data):
        result=json.loads(job_data)

        careers=result['careers']
        smartRecruiterResult=result["smartRecruiterResult"]
        dept_name=[]
        country=[loc for loc in smartRecruiterResult.keys()]

        dept_name=[]
        title=[]
        location=[]
        description=[]
        qual=[]
        job_type=[]
        postedby=[]

        ref_URL=[]
        for loc in smartRecruiterResult:
            if loc !='all':
                for i in range(len(smartRecruiterResult[loc]['content'])):
                    ref_URL.append(smartRecruiterResult[loc]['content'][i]['ref'])
        return ref_URL
                    
    def remove_unicode_chars(self, text):
        pattern = r'\\u....'
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    
    def cleanse_text(self, text):
        text = text.replace('&amp;', 'and')
        text = self.remove_unicode_chars(text)

        if '<li>' not in text and '<p>' in text:
            items_list = []
            items = re.findall(r'<p>(.*?)</p>', text)

            for item in items:
                item = item.strip('-').strip()
                items_list.append(item)
            return items_list

        if '<li><p>' in text:
            items = re.findall(r'<li><p>(.*?)</p></li>', text)
            return items

        items = re.findall(r'<li>(.*?)</li>', text)
        return items

    
    def fetch_links(self, link):

        cus1, cus2, cus3 = None, None, None
        html = urllib.request.urlopen(link)
        soup = BeautifulSoup(html, 'html.parser')
        jobs = json.loads(str(soup))
        department = ''
        title = ''
        description = ''
        qualification = ''
        job_type = ''
        postedBy = ''
        locations = ''
        title = (jobs['name'])
        try:
            cus1 = jobs['customField'][0]['fieldLabel']
            cus2 = jobs['customField'][1]['fieldLabel']
            cus3 = jobs['customField'][2]['fieldLabel']
        except:
            pass
        locations += jobs['location']['city']+', '

        if cus1 == 'Department':
            department = jobs['customField'][0]['valueLabel']
            if cus2 == 'Country':
                locations += jobs['customField'][1]['valueLabel']
            elif cus3 == 'Country':
                locations += jobs['customField'][2]['valueLabel']
        elif cus2 == 'Department':
            department = (jobs['customField'][1]['valueLabel'])
            if cus1 == 'Country':
                locations += jobs['customField'][0]['valueLabel']
            elif cus3 == 'Country':
                locations += jobs['customField'][2]['valueLabel']
        elif cus3 == 'Department':
            department = (jobs['customField'][2]['valueLabel'])
            if cus2 == 'Country':
                locations += jobs['customField'][1]['valueLabel']
            elif cus1 == 'Country':
                locations += jobs['customField'][0]['valueLabel']

        description = self.cleanse_text(jobs['jobAd']['sections']['jobDescription']['text'])
        qualification = self.cleanse_text(jobs['jobAd']['sections']['qualifications']['text'])
        
        job_type = (jobs['typeOfEmployment']['label'])
        postedBy = ' '
        d = {'title': title,
            'location': locations,
             'description': description,
             'qualification': qualification,
             'job_type': job_type,
             'postedBy': postedBy
            }
        if department not in self.df:
            self.df[department] = [d]
        else:
            self.df[department].append(d)
        
    def parallel_computing(self, ref_URL):
        threads = [threading.Thread(target=self.fetch_links, args=(url,))
                   for url in ref_URL]

        for t in threads:
            t.start()
       
    def to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.df, f)
            
jobpostings=Solution()
job_data = jobpostings.preprocessing()
ref_URL = jobpostings.url_extraction(job_data)
jobpostings.parallel_computing(ref_URL)
jobpostings.to_json('final_json1.json')

print("File created successfully")

File created successfully
